In [2]:
%cd ../scripts 
from address import addressToCords
import pandas as pd

/mnt/c/Users/ohanr/OneDrive/Unimelb/3rd Year/MAST30034/generic-real-estate-consulting-project-group-25/scripts


In [ ]:
listing = pd.read_csv("../data/curated/cleaned_property_listing_data.csv")
listing["Coordinates"] = None
listing["Latitude"] = None
listing["Longitude"] = None

In [ ]:
#Dividing the dataframe into chunks to prevent loss of progress if the Nominatium (API) server goes down
n = 100
start = 0
end = 100
while end < len(listing):
    print(start)
    try: 
        listing["Coordinates"].iloc[start: end] = listing["address"].iloc[start:end].apply(lambda x: addressToCords(x))
        listing["Latitude"].iloc[start:end] = listing["Coordinates"].iloc[start:end].apply(lambda x: x[0])
        listing["Longitude"].iloc[start:end] = listing["Coordinates"].iloc[start:end].apply(lambda x: x[1])
        listing.to_csv("../data/curated/listings_w_cords.csv")
    except:
        break
    start = end 
    end += n

Taking care of addresses that could not be found

In [3]:
listing = pd.read_csv("../data/curated/listings_w_cords.csv")

In [17]:
import requests
import re
address = "9 Killeen, Ballarat North VIC 3350"
address = re.sub("Drive||Street||Avenue||Road", "", address)
address = re.sub("^.+\/", "", address)
address = re.sub("[0-9]+\-", "", address) 
query = re.sub(" ", "+", address)
query = re.sub(",", "%2C", query)
print(query)
url = f"https://nominatim.openstreetmap.org/search?q={query}&format=json&limit=1"
location = requests.get(url).json()
location

9+Killeen%2C+Ballarat+North+VIC+3350


[]

Backup

In [ ]:
from geopy.geocoders import Nominatim
import re
import requests
def addressToCords(address: str):    
    query = re.sub(" ", "+", address)
    query = re.sub(",", "%2C", query)
    query = re.sub("\/", "%2F", query)
    url = f"https://nominatim.openstreetmap.org/search?q={query}&format=json&limit=1"
    location = requests.get(url).json()
    if location == []:
        try:
            # Getting rid of unit number
            query = re.sub("^.+%2F", "", query)
            url = f"https://nominatim.openstreetmap.org/search?q={query}&format=json&limit=1"
            location = requests.get(url).json()
            return location[0]["lat"], location[0]["lon"]
        except:
            # Only using one block as a reference 
            try:
                query = re.sub("[0-9]+\-", "", query)
                url = f"https://nominatim.openstreetmap.org/search?q={query}&format=json&limit=1"
                location = requests.get(url).json()
                return location[0]["lat"], location[0]["lon"]
            except AttributeError:
                return "Requires Manual Inspection"        

    return location[0]["lat"], location[0]["lon"]
 